# SVM: Scikit-learn sobre dataset Labeled Faces in the Wild

## O que vamos fazer?
- Descarregar e analisar o dataset Labeled Faces in the Wild (LFW). 
- Pré-processar o dataset.
- Formar um modelo de classificação multiclasse com SVM por validação cruzada. 
- Avaliar a precisão do modelo e representá-la graficamente

O dataset Labeled Faces in the Wild (LFW) é um dos datasets mais usados como exemplo e para avaliar os nossos modelos e algoritmos. É um dataset muito curioso: é uma coleção de fotos de rostos de famosos (em formato JPEG) com 13 233 exemplos, 5 749 classes (rostos de famosos) e 5 828 características (pixeis).

Toda a informação está na sua página de internet oficial: http://vis-www.cs.umass.edu/lfw/

Habitualmente usamos o dataset em 2 tipos de tarefas ou modelos diferentes de classificação:
- Verificação de faces: num conjunto de 2 fotografias, um modelo de classificação binária deve prever se é a mesma pessoa.
- Reconhecimento facial: numa fotografia, o modelo deve classificá-la como pertencendo a um dos famosos identificados (classes).


A sua tarefa será criar um modelo SVM para resolver a tarefa de reconhecimento facial, otimizar os seus hiper-parâmetros por validação cruzada e avaliá-lo.

*Vamos formar um modelo ML que seja capaz de reconhecer rostos de famosos!*

Referências:
- [The Labeled Faces in the Wild face recognition dataset](https://scikit-learn.org/stable/datasets/index.html#labeled-faces-in-the-wild-dataset)
- [sklearn.datasets.fetch_lfw_people](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_lfw_people.html)
- [sklearn.datasets.fetch_lfw_pairs](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_lfw_pairs.html)
- [Faces recognition example using eigenfaces and SVMs](https://scikit-learn.org/stable/auto_examples/applications/plot_face_recognition.html)

In [1]:
# TODO: Importar de todos os módulos necessários para esta célula

from sklearn.datasets import fetch_lfw_people
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from time import time
from scipy.stats import loguniform
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, mean_squared_error


## Carregar o dataset LFW

Representar graficamente alguns dos exemplos e as suas classes ou números associados.

*Nota*: Cuidado com o seu tamanho! Se for um dataset demasiado grande para o seu ambiente, pode reduzir o seu tamanho escolhendo um número de exemplos mais reduzido ao acaso, reordenando aleatoriamente os exemplos para evitar ter classes/faces com um número baixo de exemplos:

In [4]:
# TODO: Carregar o dataset Digits como arrays X e Y e representar alguns dos exemplos
lfw_people = fetch_lfw_people(resize=0.3)  # redimensiona as imagens para 30% do tamanho original
# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape

print(f"Exitem {n_samples} exemplos")
print(f"A imagem tem de tamanho {h} x {w}.")

Exitem 13233 exemplos
A imagem tem de tamanho 37 x 28.


In [5]:
X = lfw_people.data

# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names

print("Tamnho do dataset:", X.shape)
print("Número de classes:", len(target_names))
print("Nome das pessoas:", target_names)

Tamnho do dataset: (13233, 1036)
Número de classes: 5749
Nome das pessoas: ['AJ Cook' 'AJ Lamas' 'Aaron Eckhart' ... 'Zumrati Juma' 'Zurab Tsereteli'
 'Zydrunas Ilgauskas']


In [4]:
X[:5]

array([[0.17385621, 0.1385621 , 0.11764706, ..., 0.08888889, 0.10196079,
        0.09803922],
       [0.62091506, 0.6405229 , 0.6745098 , ..., 0.5320261 , 0.5398693 ,
        0.5111111 ],
       [0.3137255 , 0.33594772, 0.40392157, ..., 0.7738562 , 0.6732027 ,
        0.29803923],
       [0.04313726, 0.04444445, 0.04836601, ..., 0.47320262, 0.4300654 ,
        0.35947713],
       [0.10849673, 0.10457516, 0.1124183 , ..., 0.05490196, 0.06013072,
        0.07450981]], dtype=float32)

## Pré-processar os dados

Pré-processar os dados com os métodos Scikit-learn:

- Reordená-los aleatoriamente. 
- Normalizar, se necessário.
- Dividi-los em subsets de formação e testes.

Nesta ocasião, mais uma vez, faremos a validação cruzada por K-fold.

In [6]:
# TODO: Reordenar os dados aleatoriamente

indices = np.arange(len(X))
np.random.shuffle(indices)

X = X[indices]
y = y[indices]
X[:5]

array([[0.09150327, 0.08104575, 0.15294118, ..., 0.2771242 , 0.2771242 ,
        0.275817  ],
       [0.3895425 , 0.4248366 , 0.4745098 , ..., 0.48235297, 0.3019608 ,
        0.14771242],
       [0.34379086, 0.43137255, 0.38169935, ..., 0.10196079, 0.10588235,
        0.11503268],
       [0.45228758, 0.5150327 , 0.35947713, ..., 0.15555556, 0.1633987 ,
        0.33071896],
       [0.57385623, 0.6183006 , 0.6431373 , ..., 0.06013073, 0.06405229,
        0.07058824]], dtype=float32)

In [ ]:
# TODO: Normalizar, se necessário

In [7]:
# TODO: Dividi-los em subsets de formação e testes

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

scaler = StandardScaler() # padronizar os dados para que tenham média 0 e desvio padrão 1
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

De seguida, é realizada a redução de dimensionalidade nos dados de treinamento X_train e X_test, mantendo um número especificado de componentes principais, neste caso de 150, utilizando um método de cálculo aleatório e escalando os componentes para variância unitária.

In [8]:
n_components = 150

print(
    "Extracting the top %d eigenfaces from %d faces" % (n_components, X_train.shape[0])
)
pca = PCA(n_components=n_components, svd_solver="randomized", whiten=True).fit(X_train)

eigenfaces = pca.components_.reshape((n_components, h, w))

print("Projecting the input data on the eigenfaces orthonormal basis")
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

Extracting the top 150 eigenfaces from 9924 faces
Projecting the input data on the eigenfaces orthonormal basis


In [9]:
print("Dimensão de X_train:", X_train.shape)
print("Dimensão de X_train_pca:", X_train_pca.shape)
print("\nDimensão de X_test:", X_test.shape)
print("Dimensão de X_test_pca:", X_test_pca.shape)

Dimensão de X_train: (9924, 1036)
Dimensão de X_train_pca: (9924, 150)

Dimensão de X_test: (3309, 1036)
Dimensão de X_test_pca: (3309, 150)


## Formar um modelo de classificação por SVM inicial

Para comprovar o funcionamento do nosso classificador SVC antes de o otimizar por validação cruzada, vamos formar um modelo inicial sobre o subset de formação e validá-lo sobre o subset de teste.

Recordar usar a função [decision_function_shape](https://scikit-learn.org/stable/modules/svm.html#multi-class-classification) para usar o esquema “um contra o resto” (“ovr”). 

Usar os valores por defeito de *C* e *gamma* para não influir sobre a sua regularização:

In [10]:
model = SVC(decision_function_shape='ovr')
t0 = time()
model.fit(X_train_pca, y_train)
print("done in %0.3fs" % (time() - t0))

SVC()

## Avaliar o modelo no subset de teste
Neste caso, não vamos representar a matriz de confusão, pois com 5749 classes seria demasiado grande para o analisar num gráfico:

In [ ]:
# TODO: Avaliar o modelo com o seu score () no subset de teste.
t1 = time()
y_test_pred = model.predict(X_test_pca)
print("done in %0.3fs" % (time() - t1))

In [21]:
accuracy_score(y_test, y_test_pred), len(y_test)

(0.7422360248447205, 322)

Então concluimos que, provavelmente por usar os valores por defeito de C e gamma, não obteve um bom resultado de accuracy nomeadamente de 0.74.